# 4 深度学习计算

## 4.1 模型构造

### nn.Module

In [1]:
import torch
from torch import nn
from collections import OrderedDict

用nn.Module实现多层感知机
有关super()函数的详解
super函数，一言以蔽之，就是调用父类的方法
比如下函数

```
super().__init__()
```

就是调用父类nn.Module中的init方法

In [2]:
# 使用nn.Module并重构init和forward函数，使得其变成所需的网络
class MLP(nn.Module):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Module的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享”一节将介绍的模型参数params
        super(MLP, self).__init__(**kwargs)
        # 在python3中可简写为
        # super().__init__()
        # 隐藏层是一个线性的784转256
        self.hidden = nn.Linear(784, 256)  # 隐藏层
        # 一个激活函数
        self.act = nn.ReLU()
        # 输出层是一个线性256转10
        self.output = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [3]:
# 输入一些数据看一下这个网络
X = torch.rand(2, 784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.1324, -0.0532,  0.1862, -0.1805, -0.0433,  0.1061,  0.1390, -0.2413,
          0.1090, -0.0309],
        [-0.0122, -0.0742, -0.0266, -0.1849, -0.0966,  0.0499,  0.0954, -0.3064,
          0.0956,  0.0333]], grad_fn=<AddmmBackward>)

### Sequential类

这里写了一个和Sequential同功能的类，来说明Sequential的原理

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super(MySequential, self).__init__()
        # 干了这样一件事：
        # 如果传入的是OrderedDict，那就把这个OrderDict拆开拿出里面的modules一个一个加入到self._modules
        # 如果传入的是module，则直接把module加入self._modules
        if len(args) == 1 and isinstance(args[0], OrderedDict):  # 如果传入的是一个OrderedDict
            for key, module in args[0].items():
                self.add_module(key, module)  # add_module方法会将module添加进self._modules(一个OrderedDict)
        else:  # 传入的是一些Module
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)

    def forward(self, input):
        # self._modules返回一个 OrderedDict，保证会按照成员添加时的顺序遍历成员
        # 也就是说按照序列一个接一个往前运行
        for module in self._modules.values():
            input = module(input)
        return input

In [5]:
net = MySequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
)
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1216, -0.3293, -0.2056, -0.1127,  0.0763,  0.1149, -0.1661, -0.1267,
          0.0508,  0.1637],
        [ 0.0480, -0.2871, -0.0879, -0.2373,  0.0386,  0.0728, -0.3274, -0.1584,
         -0.1157,  0.1737]], grad_fn=<AddmmBackward>)

那么如果直接用Sequential则会简单许多，效果和上面一样

In [6]:
# 直接使用Sequential
net = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
)
print(net)
net(X)

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.0341,  0.1151,  0.0803, -0.4828, -0.0111,  0.0296, -0.0164,  0.0306,
         -0.1063,  0.0097],
        [ 0.0460,  0.1324,  0.1001, -0.3929,  0.0462,  0.0108, -0.0527,  0.0070,
         -0.0309, -0.0319]], grad_fn=<AddmmBackward>)

### ModuleList

ModuleList接收一个子模块的列表作为输入，然后也可以类似List那样进行append和extend操作:

In [7]:
# modulelist
net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])
net.append(nn.Linear(256, 10))  # # 类似List的append操作
print(net[-1])  # 类似List的索引访问
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


注意它和Sequential的区别，modulelist是纯粹的堆叠，而sequential是严格的一个接一个，上一个输出就是下一个输入，大小相匹配

ModuleList的出现只是让网络定义前向传播时更加灵活，见下面官网的例子。

In [8]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(10)])

    def forward(self, x):
        # ModuleList can act as an iterable, or be indexed using ints
        for i, l in enumerate(self.linears):
            x = self.linears[i // 2](x) + l(x)
        return x

另外，ModuleList不同于一般的Python的list，加入到ModuleList里面的所有模块的参数会被自动添加到整个网络中

In [9]:
# ModuleList不同于普通python的list
class Module_ModuleList(nn.Module):
    def __init__(self):
        super(Module_ModuleList, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10)])


class Module_List(nn.Module):
    def __init__(self):
        super(Module_List, self).__init__()
        # 区别在这，这里用的是普通的list
        self.linears = [nn.Linear(10, 10)]


net1 = Module_ModuleList()
net2 = Module_List()

print("net1:")
for p in net1.parameters():
    print(p.size())

print("net2:")
for p in net2.parameters():
    print(p)

net1:
torch.Size([10, 10])
torch.Size([10])
net2:


### ModuleDict类

和ModuleList一样，ModuleDict实例仅仅是存放了一些模块的字典，并没有定义forward函数需要自己定义。同样，ModuleDict也与Python的Dict有所不同，ModuleDict里的所有模块的参数会被自动添加到整个网络中。

In [10]:
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10)  # 添加
print(net['linear'])  # 访问
print(net.output)
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


### 一个稍微复杂的模型

In [11]:
# 一个稍微复杂的模型
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)

        self.rand_weight = torch.rand((20, 20), requires_grad=False)  # 不可训练参数（常数参数）
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常数参数，以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)

        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)
        # 控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()


X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)


class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU())

    def forward(self, x):
        return self.net(x)


net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

X = torch.rand(2, 40)
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)
Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(-11.4125, grad_fn=<SumBackward0>)